<a href="https://colab.research.google.com/github/elakurthyshivani/GFG-Articles-Summarizer/blob/dev%2Fbuilding-dataset/dataset/BuildingDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install packages if not yet installed**

In [1]:
import sys

!{sys.executable} -m pip install bs4 # BeautifulSoup
!{sys.executable} -m pip install opendatasets # OpenDatasets
!{sys.executable} -m pip install pyspark # PySpark

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=249f8f3908ed15e67d55baa75163c8003c3f66678c00b028aa3178adea2c582b
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=c858fd886ace9d825d85c2936040b19c1c78d348020767ec6442a6a05e0cadad
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


## **Reading the dataset**

**1.** Create a file `kaggle.json` and save your Kaggle username and API key. This will be used to download the dataset from Kaggle.

**2.** The URL of the dataset is [https://www.kaggle.com/datasets/ashishjangra27/geeksforgeeks-articles](https://www.kaggle.com/datasets/ashishjangra27/geeksforgeeks-articles "GeeksForGeeks Articles Dataset"). Using `opendatasets` package, download the dataset. Step 1 is required in order for this to automatically take in your username and API key.

**3.** Create a Spark Session to start working with PySpark.

**4.** Read the downloaded dataset.

In [2]:
import json
import opendatasets as od
from pyspark.sql import SparkSession

In [3]:
# Creating kaggle.json file.
with open("kaggle.json", "w") as kaggleFile:
    kaggleFile.write(json.dumps({"username":"shivanielakurthy", "key":"da7b4ae4bd1b770cb8b74d3990fc7f43"}))

In [4]:
# Downloading the dataset.
od.download("https://www.kaggle.com/datasets/ashishjangra27/geeksforgeeks-articles")

100%|██████████| 1.31M/1.31M [00:00<00:00, 63.8MB/s]

In [5]:
# Create a Spark Session.
spark=SparkSession.builder.config('spark.app.name', 'geeks_for_geeks_articles').getOrCreate()

In [6]:
# Reading the dataset.
articles=spark.read.option('header', True)\
          .option('inferSchema', True)\
          .csv(r"geeksforgeeks-articles/articles.csv")
articles.show(5, truncate=False)

+--------------------------------------------+----------------+------------+---------------------------------------------------------------------------+--------+
|title                                       |author_id       |last_updated|link                                                                       |category|
+--------------------------------------------+----------------+------------+---------------------------------------------------------------------------+--------+
|5 Best Practices For Writing SQL Joins      |priyankab14     |21 Feb, 2022|https://www.geeksforgeeks.org/5-best-practices-for-writing-sql-joins/      |easy    |
|Foundation CSS Dropdown Menu                |ishankhandelwals|20 Feb, 2022|https://www.geeksforgeeks.org/foundation-css-dropdown-menu/                |easy    |
|Top 20 Excel Shortcuts That You Need To Know|priyankab14     |17 Feb, 2022|https://www.geeksforgeeks.org/top-20-excel-shortcuts-that-you-need-to-know/|easy    |
|Servlet – Fetching Result  

## **Dropping rows with null values**

In [7]:
articles=articles.dropna()

## **Setup logging**

In [41]:
import logging
logging.basicConfig(filename='buildingdataset.log', level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')

## **Scrap text from the URL to get article content**

In [40]:
from bs4 import BeautifulSoup
from pyspark.sql.functions import lit, col, udf
import requests

In [42]:
# Add new column to save the scrapped text from the URLs.
articles=articles.withColumn("text", lit(""))
articles.show(5, truncate=False)

+--------------------------------------------+----------------+------------+---------------------------------------------------------------------------+--------+----+
|title                                       |author_id       |last_updated|link                                                                       |category|text|
+--------------------------------------------+----------------+------------+---------------------------------------------------------------------------+--------+----+
|5 Best Practices For Writing SQL Joins      |priyankab14     |21 Feb, 2022|https://www.geeksforgeeks.org/5-best-practices-for-writing-sql-joins/      |easy    |    |
|Foundation CSS Dropdown Menu                |ishankhandelwals|20 Feb, 2022|https://www.geeksforgeeks.org/foundation-css-dropdown-menu/                |easy    |    |
|Top 20 Excel Shortcuts That You Need To Know|priyankab14     |17 Feb, 2022|https://www.geeksforgeeks.org/top-20-excel-shortcuts-that-you-need-to-know/|easy    |    

In [43]:
# Define a User Defined Function to scrap text.
def scrapText(link):
    try:
        page=requests.get(link).text
        parser=BeautifulSoup(page, "html.parser")
        return parser.find("div", class_="text").get_text()
    except Exception as err:
        logging.error(f"ScrapText error ({link}) : {err}")
        return ""

scrapTextUDF=udf(scrapText)

In [44]:
# Apply the UDF to text column using the link column.
articles.withColumn("text", scrapTextUDF(articles["link"])).show(5)

+--------------------+----------------+------------+--------------------+--------+--------------------+
|               title|       author_id|last_updated|                link|category|                text|
+--------------------+----------------+------------+--------------------+--------+--------------------+
|5 Best Practices ...|     priyankab14|21 Feb, 2022|https://www.geeks...|    easy|SQL (Structured Q...|
|Foundation CSS Dr...|ishankhandelwals|20 Feb, 2022|https://www.geeks...|    easy|Foundation CSS is...|
|Top 20 Excel Shor...|     priyankab14|17 Feb, 2022|https://www.geeks...|    easy|Although many of ...|
|Servlet – Fetchin...| nishatiwari1719|17 Feb, 2022|https://www.geeks...|    easy|Servlet is a simp...|
|    Suffix Sum Array|        rohit768|21 Feb, 2022|https://www.geeks...|    easy|Suffix Sum ArrayG...|
+--------------------+----------------+------------+--------------------+--------+--------------------+
only showing top 5 rows



In [ ]:
# Stop the spark session.
spark.stop()